In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
from fastai.text import *
from PIL import Image, ImageDraw, ImageOps, ImageFont, ImageEnhance, ImageFilter, ImageChops
import pdb
import textwrap

In [ ]:
PATH = Path('data/fonts/')

In [ ]:
# img_path = PATH/'fonts_resize'
# df = pd.read_csv(PATH/'edited_font.csv')

In [ ]:
#1050 width
fonts = {'Indie Flower':29, 'Dancing Script':35, 'Pacifico':31, 'Shadows Into Light':33, 'Satisfy':33, 'Courgette':28,
 'Caveat':38, 'Butterfly Kids':43, 'Kaushan Script':32, 'Great Vibes':40, 'Cookie':40, 'Tangerine':49,
 'Gloria Hallelujah':25, 'Sacramento':38, 'Marck Script':30, 'Gochi Hand':30, 'Bad Script':33, 'Yellowtail':36,
 'Parisienne':34, 'Schoolbell':31, 'Homemade Apple':22, 'Allura':37, 'Damion':35, 'Alex Brush':36,
 'Covered By Your Grace':35, 'Italianno':48, 'Mr Dafoe':37, 'Nothing You Could Do':26, 'Pinyon Script':32,
 'Niconne':37, 'Nanum Pen Script':36, 'Yesteryear':39, 'Reenie Beanie':34, 'Leckerli One':27, 'Arizonia':33,
 'Just Another Hand':61, 'Coming Soon':27, 'Rochester':39, 'Grand Hotel':42,
 'Nanum Brush Script':39, 'Norican':37, 'Annie Use Your Telescope':37, 'Sue Ellen Francisco':47,
 'Herr Von MuellerHoff':49, 'Mr De Haviland':51, 'Calligraffitti':34, 'Kristi':50, 'Sofia':28, 'Aladin':38,
 'The Girl Next Door':26, 'Give You Glory':27, 'Cedarville Cursive':27, 'Monsieur La Doulaise':36, 'Montez':40,
 'Qwigley':53, 'Rouge Script':41, 'La Belle Aurore':29, 'Bilbo Swash Caps':45, 'Crafty Girls':24,
 'Waiting for the Sunrise':32, 'Meddon':20, 'Aguafina Script':44, 'Loved By The King':39, 'Vibur':35,
 'Just Me Again Down Here':42, 'Princess Sofia':37, 'Dawning Of A New Day':34, 'Mrs Saint Delafield':46,
 'Over The Rainbow':27, 'Swanky and Moo Moo':31, 'Engagement':51, 'Mr Bedfort':33,
 'Lovers Quarrel':54, 'Condiment':37, 'Charmonman':31, 'League Script':29, 'Bilbo':47, 'Gaegu':29, 'Stalemate':58,
 'Julee':32, 'Jim Nightshade':43, 'Miss Fajardose':54, 'Ruthie':44, 'Dokdo':31, 'Mrs Sheppards':37, 'Dr Sugiyama':39,
 'Felipa':34, 'Devonshire':42, 'East Sea Dokdo':43, 'Gamja Flower':35, 'Hi Melody':34, 'Ruge Boogie':40,
 'Sunshiney':37, 'Romanesco':53, 'Lakki Reddy':28}

# Sources

## Combinations

In [ ]:
a = pd.read_csv(PATH/'knot.csv', usecols=['filename', 'label'])
# b = pd.read_csv(PATH/'imdb_unsup_21k_debug.csv', usecols=['filename', 'label'])
c = pd.read_csv(PATH/'adrift.csv', usecols=['filename', 'label'])
d = pd.read_csv(PATH/'zane.csv', usecols=['filename', 'label'])
e = pd.read_csv(PATH/'american.csv', usecols=['filename', 'label'])
f = pd.read_csv(PATH/'age.csv', usecols=['filename', 'label'])
g = pd.read_csv(PATH/'room.csv', usecols=['filename', 'label'])
df = pd.concat([a,c,d,e,f,g], ignore_index=True)
len(df)

In [ ]:
a = pd.read_csv(PATH/'imdb.csv')
b = pd.read_csv(PATH/'wiki.csv')
df = pd.concat([a,b], ignore_index=True)
len(df)

In [ ]:
# add num_lines to df
lines = df.label.apply(lambda x: len(x.split('\n'))) 
df['num_lines'] = lines.astype('int32')
df.head()

In [ ]:
df.sort_values('num_lines', inplace=True)
df.reset_index(inplace=True, drop=True)

In [ ]:
df.to_csv(PATH/'imdb_wiki_combo.csv', index=False)

In [ ]:
new = df[df.num_lines < 7]
len(new)

In [ ]:
new.to_csv(PATH/'imdb_wiki_combo<7.csv', index=False)

### Move Images to new folder based on criteria

In [ ]:
lines = new.label.apply(lambda x: len(x.split('\n'))) 
new['num_lines'] = lines.astype('int32')
new.head()

In [ ]:
lines = new.label.apply(lambda x: sum(1 for c in x if c.isupper())) 
new['num_caps'] = lines.astype('int32')
new.head()

In [ ]:
ten_lines = new[new.num_caps > 9]
len(ten_lines)

In [ ]:
def move_images(fnames, src, targ):
    for fname in fnames:
        try:
            os.rename(src/fname, targ/fname)
        except FileNotFoundError:
            pass

In [ ]:
src_path = PATH/'combo'
targ_path = PATH/'combo_test'
os.makedirs(targ_path, exist_ok=True)

In [ ]:
move_images(ten_lines.filename.values, src_path, targ_path)

### Populate df with only files left in directory

In [ ]:
len(df)

In [ ]:
fnames = os.listdir(img_path)
len(fnames)

In [ ]:
df = df[df.filename.isin(fnames)]
len(df)

### Remove files not in df

In [ ]:
files = []

In [ ]:
for filename in progress_bar(os.listdir(PATH/'fonts'), total=81256):
    if filename not in new.filename.values:
        files.append(filename)
        os.rename(str(PATH/'fonts'/filename), str(PATH/'overflow_fonts'/filename))  # move
#         os.remove(filename)

In [ ]:
# files = []
for fname in progress_bar(mins.filename.values):
#     files.append(fname)
    os.rename(str(PATH/'fonts'/fname), str(PATH/'fonts_removed'/fname))

In [ ]:
len(files)

In [ ]:
row = mins[mins.filename == 'zane_1508_0.png']
print(mins.loc[row.index[0]]['label'])

## IMDB

In [ ]:
pre = 'imdb_'
img_path = PATH/'imdb_wiki_combo'
csv_path = PATH/'imdb.csv'

df = pd.read_csv(Path('data/fonts/texts/imdb.csv'))

## Wiki103

In [ ]:
pre = 'wiki_'
img_path = PATH/'imdb_wiki_combo'
csv_path = PATH/'wiki.csv'

df = pd.read_csv(Path('data/fonts/texts/wiki103.csv'))

### Remove non ASCII chars and resave txt file

In [ ]:
with open(text_path) as file:  
    text = file.read()

text = re.sub(r'[^\x00-\x7F]+','', text)

with open(str(text_path), "w") as text_file:
    print(text, file=text_file)

## Arthur - Cast Adrift

In [ ]:
img_path = PATH/'adrift'
csv_path = PATH/'adrift.csv'
text_path = Path('data/texts/cast_adrift.txt')

In [ ]:
# combine
df = pd.read_csv(csv_path)
len(df)

In [ ]:
df.to_csv(csv_path, columns=['filename', 'label'], index=False)

## Grey - Betty Zane

In [ ]:
img_path = PATH/'zane'
csv_path = PATH/'zane.csv'
text_path = Path('data/texts/betty_zane.txt')

In [ ]:
# combine
df = pd.read_csv(csv_path)
len(df)

In [ ]:
df.to_csv(csv_path, columns=['filename', 'label'], index=False)

## James - The American

In [ ]:
img_path = PATH/'american'
csv_path = PATH/'american.csv'
text_path = Path('data/texts/the_american.txt')

In [ ]:
# combine
# a = pd.read_csv(PATH/'american0.csv', usecols=['filename', 'label'])
# b = pd.read_csv(PATH/'american.csv', usecols=['filename', 'label'])
# df = pd.concat([a,b], ignore_index=True)

df = pd.read_csv(csv_path)
len(df)

In [ ]:
!ls {img_path} | wc -l

In [ ]:
df.to_csv(csv_path, columns=['filename', 'label'], index=False)

## Wharton - Age of Innocence

In [ ]:
img_path = PATH/'age'
csv_path = PATH/'age.csv'
text_path = Path('data/texts/age_of_innocence.txt')

In [ ]:
# combine
# a = pd.read_csv(PATH/'age0.csv', usecols=['filename', 'label'])
# b = pd.read_csv(PATH/'age.csv', usecols=['filename', 'label'])
# df = pd.concat([a,b], ignore_index=True)

df = pd.read_csv(csv_path)
len(df)

In [ ]:
!ls {img_path} | wc -l

In [ ]:
df.to_csv(csv_path, columns=['filename', 'label'], index=False)

## Forster - Room with a View

In [ ]:
img_path = PATH/'room'
csv_path = PATH/'room.csv'
text_path = Path('data/texts/room_with_a_view.txt')

In [ ]:
# combine
# a = pd.read_csv(PATH/'room0.csv', usecols=['filename', 'label'])
# c = pd.read_csv(PATH/'room1.csv', usecols=['filename', 'label'])
# b = pd.read_csv(PATH/'room.csv', usecols=['filename', 'label'])
# df = pd.concat([a,c,b], ignore_index=True)

df = pd.read_csv(csv_path)
len(df)

In [ ]:
!ls {img_path} | wc -l

In [ ]:
df.sort_values('filename', inplace=True)
df.reset_index(inplace=True, drop=True)

In [ ]:
df.to_csv(csv_path, columns=['filename', 'label'], index=False)

## Shaw - Irrational Knot

In [ ]:
img_path = PATH/'knot'
csv_path = PATH/'knot.csv'
text_path = Path('data/texts/irrational_knot.txt')

In [ ]:
# combine
# a = pd.read_csv(PATH/'shaw_knot.csv')
# b = pd.read_csv(PATH/'shaw_knot1.csv')
# df = pd.concat([a,b], ignore_index=True)

df = pd.read_csv(csv_path)
len(df)

In [ ]:
df.sort_values('filename', inplace=True)
df.reset_index(inplace=True, drop=True)

In [ ]:
df.to_csv(csv_path, columns=['filename', 'label'], index=False)

# Text Data

In [ ]:
with open(text_path) as file:  
    text = file.read()

In [ ]:
text = text.replace('\n',' ')
lines = text.split('  ')
len(lines)

In [ ]:
df = pd.DataFrame({'text': lines})

In [ ]:
# remove small lines
df['char_len'] = df.text.map(len)
df = df[df['char_len'] > 15]
df.sort_values('char_len', inplace=True)
df.head(5)

In [ ]:
import textwrap

# truncate text len to 1000 w/ textwrap
df['text'] = df.text.map(lambda x: textwrap.wrap(x, width=700, replace_whitespace=False)[0])
df['char_len'] = df.text.map(len)

In [ ]:
df.sort_index(inplace=True)

In [ ]:
texts = df.text.values

In [ ]:
len(texts)

# Background

In [ ]:
# random image from folder
def rand_from_folder(f_path):
    files = [f for f in os.listdir(f_path) if not f.startswith(".")]  # remove .DS_Store and other hidden files
    fname = random.choice(files)
    return Image.open(f_path/fname).convert('L')

In [ ]:
def skew_pixels(im, brightness=0.5, sharpness=-0.5):
    '''increase brightness to lighten darks
       decrease sharpness to spread values out
    '''
    enhance = ImageEnhance.Brightness(im)
    im = enhance.enhance(1.0 + brightness)  # increase brightness
    enhance = ImageEnhance.Sharpness(im)
    im = enhance.enhance(1.0 + sharpness)  # decrease sharpness
    return im

In [ ]:
def overlay_bg(im, bg_img=None):
    if bg_img is None:
        bg_img = rand_from_folder(Path('data/IAM_handwriting/paragraphs'))
    blur = 70 #random.randint(70,150) 
    bg = bg_img.filter(ImageFilter.GaussianBlur(blur))
    # random flips
    choices = [Image.FLIP_LEFT_RIGHT, Image.FLIP_TOP_BOTTOM, Image.ROTATE_90, Image.ROTATE_180, Image.ROTATE_270]
    bg.transpose(random.choice(choices))

    return ImageChops.darker(im, bg.resize(im.size))
#     diff = PIL.ImageChops.difference(im, bg.resize(im.size))
#     return PIL.ImageChops.subtract(im, diff)

In [ ]:
def poisson_noise(image, mod=1.5):
    image = np.array(image)
    # Determine unique values in image & calculate the next power of two
    vals = len(np.unique(image))
    vals = mod ** np.ceil(np.log2(vals))

    # Generating noise for each unique value in image.
    out = np.random.poisson(image * vals) / float(vals)
    return Image.fromarray(out.astype('uint8'), mode='L')

In [ ]:
def crop_image(im):    
    bg = Image.new(im.mode, im.size, im.getpixel((0,0)))
    diff = ImageChops.difference(im, bg)
    diff = ImageChops.add(diff, diff, 2.0, -100)
    box = diff.getbbox()
    return im.crop(box)

In [ ]:
def pad_image(im, pad_range=(10,50)):
    "adjusted to minimize large ratio differences"
    row,col = im.size
    bg_color = im.getpixel((0,0))
    l = int(random.randint(*pad_range) * col/row)
    u = int(random.randint(*pad_range) * row/col)
    r = int(random.randint(*pad_range) * col/row)
    b = int(random.randint(*pad_range) * row/col)
    #print(l,u,r,b)
    return ImageOps.expand(im, border=(l,u,r,b), fill=bg_color)

In [ ]:
def add_background(img, crop=True, pad=True, skew=True, bg=True, poisson=True):
    if crop: img = crop_image(img)
    if pad: img = pad_image(img)
    if skew: img = skew_pixels(img)
    if bg: img = overlay_bg(img)
    if poisson: img = poisson_noise(img, 1.8)
    return img

# PIL generate images

In [ ]:
class FontGenerator:
    def __init__(self, texts, fonts, font_path=None, size=(1050,1000), xy=(10,10)):
        self.texts = texts
        self.fonts = fonts
        self.font_path = font_path or "./data/fonts/font_files/"
        self.size = size
        self.xy = xy
        self.results = []
        
    def create_images(self, total, img_path, pre='', offset=0):
        for idx in progress_bar(range(offset, total)):
            font = self.font_type()
            t,nl = self.random_text()
            image = self.text_image(t, font)
            image = add_background(image)
            fname = pre+'{:05d}'.format(idx)+'.png'
            image.save(str(img_path/fname))
            o = {'filename':fname, 'label':t, 'font_name':font.getname()[0], 'font_size':font.size, 'num_lines': nl}
            self.results.append(o)

    def save_csv(self, csv_path):
        pd.DataFrame(self.results).to_csv(csv_path, index=False)
        
    def font_type(self, name=None, size=None):
        name = name or random.choice(list(self.fonts.keys()))
        size = size or int(self.fonts[name]) - random.randint(0,5)
        name = name.replace(' ','')+'-Regular.ttf'
        return ImageFont.truetype(self.font_path+name, size)

    def text_image(self, text, font):
        bg = random.randrange(240,255)
        image = Image.new("L", self.size, color=bg)
        spacing = random.randrange(10,40)
        color = random.randrange(10,80)
        draw = ImageDraw.Draw(image)
        draw.text(self.xy, text, fill=color, font=font, spacing=spacing)
        return image

    def random_text(self):
        t = np.random.choice(self.texts)
        t = textwrap.fill(t, width=60-random.randint(0,10))
        t = t.split('\n')[:10]      # only keep maximum 10 lines    
        nl = len(t)
        return '\n'.join(t), nl

In [ ]:
fg = FontGenerator(df.text.values, fonts)

In [ ]:
img_path, csv_path

In [ ]:
# !rm -rf {img_path}
# os.makedirs(img_path, exist_ok=True)

In [ ]:
fg.create_images(50000, img_path, pre)
fg.save_csv(csv_path)

In [ ]:
csv = pd.read_csv(csv_path)
csv.head()

# Google Fonts

In [ ]:
img_path = PATH/'test'
csv_path = PATH/'test.csv'

In [ ]:
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [ ]:
# def set_font_size(driver, actions, value):
#     slider = driver.find_element_by_class_name('md-track-container') # whole container not just the slider
#     width = slider.size['width']
#     s_max,s_min = 300,8
#     step = width / (s_max - s_min)
#     val = int(step * (value - s_min + 1))
#     actions.click_and_hold(slider).move_by_offset(-width//2,0).move_by_offset(val,0).release().perform()

In [ ]:
class SeleniumFontGenerator:
    def __init__(self, opts, fonts):
        self.options = opts
        self.fonts = fonts
        self.results = []
        self.driver, self.actions, self.wait = None, None, None
        
    def new_driver(self):
        self.driver, self.actions, self.wait = self.driver_setup()
    
    def page_setup(self, font_name=None, font_size=None, pad_top=10):
        self.font_name = font_name or random.choice(list(self.fonts.keys()))
        self.font_size = font_size or int(self.fonts[name]) - random.randint(1,5)
#         self.letter_spacing = random.randrange(-0.0, , step=0.01)
        self.pad_top = pad_top
        self.font_input = self.set_font()
        self.text_form = self.wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'font-preview-text')))
        #self.set_text_settings()  need to set text settings after inputting text...
                
    def save_image(self, t, img_path, fname):
        if '\n' not in t:
            t = textwrap.fill(t, width=60-random.randint(0,10))
            # this is here for individual verification purposes...
        t = t.split('\n')[:10]      # only keep maximum 10 lines    
        nl = len(t)
        t = '\n'.join(t)

        self.input_text(t)
        self.set_text_settings()
        
        # fix unintentional line breaks text
        while not self.compare_heights(nl):
            self.font_size -= 5
            self.set_text_settings()

        # fix overflow text
        while not self.hide_cursor():                  
            t = t.split('\n')[:-2]  # remove last 2 lines at a time
            nl = len(t)
            t = '\n'.join(t)
            self.input_text(t)
            self.set_text_settings()

        self.text_form.screenshot(str(img_path/fname))
        out = {'filename':fname, 'label':t, 'font_name':self.font_name, 'font_size':self.font_size,
               'num_lines': nl, 'line_height':self.line_height(), 'height':self.actual_height()}
        self.results.append(out)
        
    def set_font(self):
        font_input = self.wait.until(EC.element_to_be_clickable((By.ID, 'input_4')))
        font_input.clear()
        font_input.send_keys(self.font_name)
        return font_input
    
    def input_text(self, text):
        self.text_form.clear()
        self.text_form.send_keys(text)
        
    def compare_heights(self, nl):
        exp = max(132, nl*self.font_size*self.line_height()+self.pad_top)
        act = self.actual_height()
        return (exp>=act and act<=940)  
    
    def hide_cursor(self):
        try:
            self.font_input.send_keys("")
            return True
        except:
            return False
    
    def set_text_settings(self):
        tf = self.text_form
        self.driver.execute_script(f"arguments[0].style.fontSize = '{self.font_size}px';", tf)
        self.driver.execute_script(f"arguments[0].style.lineHeight = {self.line_height()};", tf)
        self.driver.execute_script(f"arguments[0].style.paddingTop = '{self.pad_top}px';", tf)
        self.driver.execute_script(f"arguments[0].style.letterSpacing = '-.08em';", tf)

    def line_height(self): return -0.01*self.font_size+1.55 + 0.6
    def actual_height(self): return self.text_form.size['height']
    def close(self): self.driver.close()
    
    def driver_setup(self):
        try:
            self.close()
        except:
            pass

        driver = webdriver.Chrome(options=self.options)
        driver.get('https://fonts.google.com/?category=Handwriting')
        #view as list
        driver.find_element_by_class_name('font-preview-layout-toggle-button').click()
        # remove scroll-to-top button
        scroll_btn = driver.find_element_by_class_name('scroll-top-button-fab')
        driver.execute_script("arguments[0].parentNode.removeChild(arguments[0]);", scroll_btn)
        # remove footer
        footer = driver.find_element_by_id('footer')
        driver.execute_script("arguments[0].parentNode.removeChild(arguments[0]);", footer)
        return driver, ActionChains(driver), WebDriverWait(driver, 5)

## bulk

In [ ]:
options = Options()
options.add_argument("--headless")  
options.add_argument("--window-size=1050,1000")   #("--window-size=%s" % WINDOW_SIZE)

In [ ]:
def save_images(total, texts, num, img_path, csv_path, pre='', offset=0):
    sfg = SeleniumFontGenerator(options, fonts)
    try:
        for idx in progress_bar(range(offset, total)):
            if sfg.driver is None: sfg.new_driver()
            # reset driver every now and then to keep from getting throttled
            if idx%10 == 0: sfg.new_driver()
            sfg.page_setup()
            ts = np.random.choice(texts, num)
            for i,t in enumerate(ts):
                fname = pre+'{:04d}'.format(idx)+f"_{i}.png"
                sfg.save_image(t, img_path, fname)
    except:
        print(sys.exc_info())
        sfg.driver.save_screenshot(str(Path('data/fonts/fail.png')))
    finally:
        pd.DataFrame(sfg.results).to_csv(csv_path, index=False)
        return sfg

In [ ]:
csv_path = PATH/'room1.csv'

In [ ]:
img_path, csv_path

In [ ]:
# !rm -rf {img_path}
# os.makedirs(img_path, exist_ok=True)

In [ ]:
sfg = save_images(10000, texts, 5, img_path, csv_path, pre='full_', offset=0)

In [ ]:
sfg.close()

## Individual

In [ ]:
options = Options()
# options.add_argument("--headless")
options.add_argument("--window-size=1050,1000")   #("--window-size=%s" % WINDOW_SIZE)

In [ ]:
t = "Yes, go and bathe, George. Why do you dawdle talking? Bring\nthem back to tea. Bring back some milk, cakes, honey. The\nchange will do you good. George has been working very hard\nat his office. I can't believe he's well."

In [ ]:
t = "Now, this was abominably impertinent, and she ought to have been furious. But it is sometimes as difficult to lose one's temper as it is difficult at other times to keep it. Lucy could not get cross. Mr. Emerson was an old man, and surely a girl might humour him. On the other hand, his son was a young man, and she felt that a girl ought to be offended with him, or at all events be offended before him. It was at him that she gazed before replying. Now, this was abominably impertinent, and she ought to have been furious. But it is sometimes as difficult to lose one's temper as it is difficult at other times to keep it. Lucy could not get cross. Mr. Emerson was an old man, and surely a girl might humour him. On the other hand, his son was a young man, and she felt that a girl ought to be offended with him, or at all events be offended before him. It was at him that she gazed before replying."

In [ ]:
nl = 10
t = "Now, this was abominably impertinent, and she ought to\nhave been furious. But it is sometimes as difficult to\nlose one's temper as it is difficult at other times to\nkeep it. Lucy could not get cross. Mr. Emerson was an old\nman, and surely a girl might humour him. On the other\nhand, his son was a young man, and she felt that a girl\nought to be offended with him, or at all events be\noffended before him. It was at him that she gazed before\nreplying. Now, this was abominably impertinent, and she\nought to have been furious. But it is sometimes as"
print(t)

In [ ]:
sfg = SeleniumFontGenerator(options, fonts)
sfg.new_driver()
sfg.page_setup('Romanesco', 58)
test_path = Path('data/fonts')
fname = 'test.png'
sfg.save_image(t, test_path, fname)
print(sfg.results[-1])

In [ ]:
sfg.close()

# Scripts

In [ ]:
t = "Now, this was abominably impertinent, and she\nought to have been furious. But it is sometimes as\ndifficult to lose one's temper as it is difficult\nat other times to keep it. Lucy could not get\ncross. Mr. Emerson was an old man, and surely a\ngirl might humour him. On the other hand, his son\nwas a young man, and she felt that a girl ought to\nbe offended with him, or at all events be offended\nbefore him. It was at him that she gazed before\nreplying.\nNow, this was abominably impertinent, and she\nought to have been furious. But it is sometimes as\ndifficult to lose one's temper as it is difficult\nat other times to keep it. Lucy could not get\ncross. Mr. Emerson was an old man, and surely a\ngirl might humour him. On the other hand, his son\nwas a young man, and she felt that a girl ought to\nbe offended with him, or at all events be offended\nbefore him. It was at him that she gazed before\nreplying."

## Comparing img heights w/ estimated heights by font/size

### Calculating Estimated Line Heights

In [ ]:
def set_font(wait, fontName):
    font_input = wait.until(EC.element_to_be_clickable((By.ID, 'input_4')))
    font_input.clear()
    font_input.send_keys(fontName)

In [ ]:
def determine_max_line_heights(csv_path):
    res = []
    txt = "ABC\nDEF\nGHI\nJKL\nMNO\nPQR\nSTU\nVWX"
    driver = None
    
    try:
        for name,size in progress_bar(fonts.items(), total=len(fonts)): 
            driver,actions,wait = new_driver(driver)
            set_font(wait, name)

            text_form = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'font-preview-text')))
            text_form.clear()
            text_form.send_keys(txt)

            for sz in range(size-5,size):
                set_font_size(driver, actions, sz)
                total = text_form.size['height']
                res.append({'font': name, 'size': sz, 'line_height': total//8})
    finally:
        pd.DataFrame(res).to_csv(csv_path, index=False)
        driver.close()

In [ ]:
lh

In [ ]:
determine_max_line_heights(PATH/'line_heights.csv')

In [ ]:
1.8-.1

### Calculating Actual/Expected Heights

In [ ]:
lh = pd.read_csv(PATH/'line_heights.csv')
lh.set_index(['font', 'size'], inplace=True)  #multi-index
# lh.head(10)

In [ ]:
def img_height(row):
    with Image.open(img_path/row['filename']) as img:
        return img.size[1]
    
def expected_height(row):
    try:
        h = lh.loc[row['font'], row['size']].line_height
    except TypeError:
        h = lh.loc[row['font']].values.flatten()[0]  #size is missing: choose smallest size
    nl = len(row['label'].split('\n'))
    return max(264, h * nl * 2)  # 264 is minimum height


def set_heights(df):
    heights = df.apply(lambda x: img_height(x), axis=1)  
    df['height'] = heights.astype('int32')
    
#     hgt = df.apply(lambda x: expected_height(x), axis=1)  
#     df['exp_height'] = hgt.astype('int32')
    
#     df['diff'] = df.apply(lambda row: (row['height'] - row['exp_height']), axis=1)
    
    
def taller_than_expected(df): return df[df['diff'] > 0].sort_values('diff', ascending=False)

In [ ]:
# set_heights(df)
df.head()

In [ ]:
# calculate character lengths
lgts = df.label.apply(len)  
df['char_len'] = lgts.astype('int32')

In [ ]:
more = taller_than_expected(df)
len(more)

### Remove from df

In [ ]:
df.drop(more.index.values, inplace=True)
len(df)

### Check images below minimum

In [ ]:
mins = df[(df['height'] < 265) & (df['char_len'] > 65)]
len(mins)

In [ ]:
im = PIL.Image.open(PATH/'fonts_removed'/'zane_1507_1.png')

In [ ]:
im.size

In [ ]:
img_path = PATH/'fonts_removed'

In [ ]:
def trim(im):
    bg = PIL.Image.new(im.mode, im.size, im.getpixel((0,0)))
    diff = PIL.ImageChops.difference(im, bg)
    diff = PIL.ImageChops.add(diff, diff, 2.0, -100)
    bbox = diff.getbbox()
    if bbox:
        return im.crop(bbox).size[1]
    else:
        return 264
    
def trimmed_height(row):
    with PIL.Image.open(img_path/row['filename']) as img:
        return trim(img)
    
def set_heights(df):
    heights = df.apply(lambda x: trimmed_height(x), axis=1)
    df['height'] = heights.astype('int32')
    
    nl = df.label.apply(lambda x: len(x.split('\n')))  
    df['num_lines'] = nl.astype('int32')
    
#     df['diff'] = df.apply(lambda row: (row['height'] - row['exp_height']), axis=1)
    
    
def taller_than_expected(df): return df[df['diff'] > 0].sort_values('diff', ascending=False)

In [ ]:
set_heights(mins)

In [ ]:
mins[mins['height']<10]

In [ ]:
more = taller_than_expected(df)
len(more)

## Replace overflowing images

In [ ]:
def replace_images(df, img_path):
    driver = None
    try:
        for idx,row in progress_bar(df.iterrows(), total=len(df)):
            if idx%20 == 0: driver,actions,wait = new_driver(driver)

            # set font name and size
            name = random.choice(list(fonts.keys()))
            #font_input = driver.find_element_by_id('input_4')
            font_input = wait.until(EC.element_to_be_clickable((By.ID, 'input_4')))
            font_input.clear()
            font_input.send_keys(name)
            size = fonts[name] - 10 #random.randint(1,5)
            set_font_size(driver, actions, size)

            text_form = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'font-preview-text')))
            text_form.clear()
            text_form.send_keys(t)
            # remove cursor
            font_input.send_keys("")
                # save img
                fname = pre+'{:04d}'.format(idx+offset)+f"_{i}.png"
                text_form.screenshot(str(img_path/fname))
                res.append({'filename':fname, 'label':t, 'font':name, 'size':size})
    except:
        print(sys.exc_info())
        driver.save_screenshot(str(img_path/'fail.png'))
    finally:
        pd.DataFrame(res).to_csv(csv_path, index=False)
        driver.close()

In [ ]:
img_path, csv_path

In [ ]:
# !rm -rf {img_path}
os.makedirs(img_path, exist_ok=True)

In [ ]:
save_images(1155, texts, 5, img_path, csv_path, pre='imdb_unsup_', offset=2846)

## Naive check for overflow

In [ ]:
def text_overflow(txt, text_form):
    st = "\nABCDEFGHI\nJKLMNOPQR\nSTUVWXYZ\n"
    st += txt
    num_lines = len(st.split("\n"))
    text_form.clear()
    text_form.send_keys(st)
    line_height = text_form.find_element_by_xpath("div[3]").size['height']
    total_height = text_form.size['height']
    return total_height != line_height * num_lines

In [ ]:
def check_images_for_overflow(csv_path, img_path):
    df = pd.read_csv(csv_path)
    pageFont,pageSize = None,None
    driver = None
    redo = False
    
    for idx,row in progress_bar(df.iterrows(), total=len(df)):
        if idx%20 == 0: driver,actions,wait = new_driver(driver)
            
        fname,txt,rowFont,rowSize = row['filename'],row['label'],row['font'],row['size']
        if pageFont != rowFont: set_font(wait, rowFont)
        if pageSize != rowSize: set_font_size(driver, actions, rowSize)
        
        text_form = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'font-preview-text')))
        while text_overflow(txt, text_form):
            redo = True
            set_font_size(driver, actions, rowSize-5)
        
        if redo:
            text_form.clear()
            text_form.send_keys(txt)
        
            # remove cursor
            set_font(wait, rowFont)

            new_path = str(img_path)+'_overflow/'+fname
            text_form.screenshot(new_path)
        redo = False

In [ ]:
img_path, csv_path

In [ ]:
os.makedirs(str(img_path)+'_overflow/', exist_ok=True)

In [ ]:
check_images_for_overflow(PATH/'imdb_unsup_20k_debug.csv', img_path)

# Verify

In [ ]:
# df_v = pd.read_csv(csv_path)
# len(df_v)

# df_v.tail()

In [ ]:
y = iter(range(len(csv)))

In [ ]:
i = next(y) #random.randrange(0,5000) #
row = csv.iloc[i]
im = Image.open(img_path/row.filename)
fig,ax = plt.subplots(figsize=(18,12))
ax.imshow(im)
print(row.filename)
print(row.label)